In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

import requests
import json
import os
from datetime import datetime
from datetime import timedelta
import time

import folium

In [2]:
ayer = datetime.today() + timedelta(days=-1)
for i in range(-6, 0):
    time.sleep(2)
    otro_dia = ayer + timedelta(days=i)
    #convierto la fecha de hoy en el formato que lo acepta la consulta de api
    #fecha = str(ayer.day) + '/' + str(ayer.month) + '/' + str(ayer.year) + ' 0:00:00'
    fecha = str(otro_dia.day) + '/' + str(otro_dia.month) + '/' + str(otro_dia.year) + ' 0:00:00'
    #se realiza la peticion al api para traer los datos de la fecha de hoy
    resultado = requests.get("https://www.datos.gov.co/resource/gt2j-8ykr.json?$limit=100000&$where=(fecha_reporte_web="+"'"+fecha+"')")
    #se crea una variable con el nombre con el que se guardaran los datos, en formato json
    #nombre_archivo = str(ayer.date()).replace('-', '') + '.json'
    nombre_archivo = str(otro_dia.date()).replace('-', '') + '.json'
    #se crea la ruta para guardar el archivo
    ruta_archivo = './data/data_cruda/' + nombre_archivo
    #se crea el archivo con el nombre asociado al día descargado
    with open(ruta_archivo, 'w') as fp:
        json.dump(resultado.json(), fp)

In [ ]:
cols = ['fecha_reporte_web','departamento','departamento_nom','ciudad_municipio','ciudad_municipio_nom','num_casos']
info = pd.read_csv('./data/consolidado.csv')
info.columns = cols
info.fecha_reporte_web = pd.to_datetime(info.fecha_reporte_web, dayfirst=True)

df = info[info['fecha_reporte_web'] ==max(info.fecha_reporte_web)]
df['ciudad_municipio'] = df['ciudad_municipio'].apply(lambda x: '0' * (5-len(str(x)))+str(x))

In [6]:
os.getcwd()

'/home/ceao/Documents/proyecto_covid'

In [34]:
poblacion_ciudad = gpd.read_file('./shp_mapa_municipio/nivel2/MGN_ANM_MPIOCL.dbf')
#poblacion_ciudad = poblacion_ciudad[['MPIO_CDPMP', 'STP27_PERS']].groupby('MPIO_CDPMP').sum().reset_index()

In [223]:
import pickle

with open('./assets/depto_dict.pkl', 'rb') as fp:
    prueba = pickle.load(fp)

list(prueba.keys())


['ANTIOQUIA',
 'ATLANTICO',
 'BOGOTA',
 'BOLIVAR',
 'BOYACA',
 'CALDAS',
 'CAQUETA',
 'CAUCA',
 'CESAR',
 'CORDOBA',
 'CUNDINAMARCA',
 'CHOCO',
 'HUILA',
 'GUAJIRA',
 'MAGDALENA',
 'META',
 'NARIÑO',
 'NORTE SANTANDER',
 'QUINDIO',
 'RISARALDA',
 'SANTANDER',
 'SUCRE',
 'TOLIMA',
 'VALLE',
 'ARAUCA',
 'CASANARE',
 'PUTUMAYO',
 'SAN ANDRES',
 'AMAZONAS',
 'GUAINIA',
 'GUAVIARE',
 'VAUPES',
 'VICHADA']

In [193]:
depto_dict = {}
mpio_dict = {}
for depto in df_dict['cod_depto'].unique():
    
    temp = df_dict[df_dict['cod_depto']==depto]
    depto_dict[temp['nom_depto'].unique()[0]] = depto
    mpio_dict[depto] = {i['nom_mpio']:i['MPIO_CDPMP'] for i in temp[['MPIO_CDPMP','nom_mpio']].to_dict('records')}


In [208]:
list(mpio_dict['50'].keys())

['VILLAVICENCIO',
 'ACACIAS',
 'CABUYARO',
 'CASTILLA LA NUEVA',
 'CUMARAL',
 'EL CASTILLO',
 'FUENTE DE ORO',
 'GRANADA',
 'GUAMAL',
 'LEJANIAS',
 'PUERTO GAITAN',
 'PUERTO LLERAS',
 'PUERTO RICO',
 'RESTREPO',
 'SAN JUAN DE ARAMA',
 'SAN MARTIN',
 'VISTAHERMOSA',
 'BARRANCA DE UPIA',
 'MESETAS',
 'LA URIBE',
 'PUERTO CONCORDIA',
 'PUERTO LOPEZ',
 'SAN CARLOS DE GUAROA',
 'LA MACARENA',
 'SAN JUANITO',
 'CUBARRAL',
 'EL CALVARIO',
 'MAPIRIPAN',
 'EL DORADO']

In [115]:
info['05']

{'nom_depto': 'Antioquia',
 'municipios': {'Medellin': '05001', 'Abejorral': '05002'}}

In [41]:
features_ciudad = gpd.read_file('./shp_mapa_municipio/nivel2/MGN_ANM_MPIOCL.dbf')
features_ciudad = features_ciudad[['MPIO_CDPMP', 'STP27_PERS']].groupby('MPIO_CDPMP').sum().reset_index()
features_ciudad.rename(columns={'STP27_PERS':'poblacion'}, inplace=True)

consolidado = pd.read_csv('./data/consolidado.csv')

cols=['fecha', 'cod_depto','nom_depto','MPIO_CDPMP','nom_mpio','num_casos']
consolidado.columns =cols
consolidado.fecha = pd.to_datetime(consolidado['fecha'], dayfirst=True)
consolidado['MPIO_CDPMP'] = consolidado['MPIO_CDPMP'].apply(lambda x: '0' * (5-len(str(x)))+str(x))

consolidado = consolidado.merge(features_ciudad, on='MPIO_CDPMP', how='left')
consolidado['cod_depto'] = consolidado['MPIO_CDPMP'].str[:2]

consolidado['casos_por_millon'] = np.ceil(consolidado['num_casos'] / (consolidado['poblacion'] /1000000))

df_ultimo = consolidado[consolidado['fecha']==max(consolidado['fecha'])]

print(df_ultimo[df_ultimo['nom_mpio'].isin(['BOGOTA','CARTAGENA','MEDELLIN','VILLAVICENCIO'])])

#create the dictionary for filters, dropdown and others objects


df_dict = consolidado[['cod_depto','nom_depto','MPIO_CDPMP','nom_mpio']]


           fecha cod_depto  nom_depto MPIO_CDPMP       nom_mpio  num_casos  \
28695 2021-08-25        05  ANTIOQUIA      05001       MEDELLIN        371   
28737 2021-08-25        11     BOGOTA      11001         BOGOTA        513   
28862 2021-08-25        50       META      50001  VILLAVICENCIO         17   
28976 2021-08-25        13  CARTAGENA      13001      CARTAGENA         98   

       poblacion  casos_por_millon  
28695  2372330.0             157.0  
28737  7181469.0              72.0  
28862   451212.0              38.0  
28976   887946.0             111.0  


In [35]:
poblacion_ciudad[['DPTO_CCDGO','']]

,DPTO_CCDGO,MPIO_CCDGO,MPIO_CDPMP,CLAS_CCDGO,MPIO_CLASE,VERSION,AREA,LATITUD,LONGITUD,STCTNENCUE,...,STP34_9_ED,STP51_PRIM,STP51_SECU,STP51_SUPE,STP51_POST,STP51_13_E,STP51_99_E,Shape_Leng,Shape_Area,geometry
0,05,001,05001,1,050011,2018,1.209855e+08,6.246716,-75.581713,960527.0,...,57744.0,510137.0,428083.0,508160.0,62109.0,52707.0,34873.0,1.216977,9.879336e-03,None
1,05,001,05001,2,050012,2018,1.207613e+07,6.228217,-75.539494,10462.0,...,301.0,4813.0,1294.0,2278.0,124.0,632.0,121.0,0.929962,9.861053e-04,None
2,05,002,05002,1,050021,2018,1.238698e+06,5.789315,-75.428739,3801.0,...,283.0,1582.0,1193.0,441.0,15.0,419.0,105.0,0.071662,1.010790e-04,None
3,05,021,05021,1,050211,2018,4.021252e+05,6.376061,-75.141346,1455.0,...,101.0,833.0,260.0,294.0,0.0,65.0,25.0,0.040012,3.285565e-05,None
4,05,030,05030,1,050301,2018,1.547309e+06,6.038817,-75.702041,6141.0,...,272.0,3216.0,2839.0,1495.0,26.0,422.0,93.0,0.089251,1.262863e-04,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3150,99,624,99624,2,996242,2018,2.009818e+05,4.853461,-71.038317,105.0,...,0.0,0.0,0.0,0.0,0.0,7.0,2.0,0.026521,1.633909e-05,None
3151,05,667,05667,2,056672,2018,9.441049e+03,6.352836,-74.881349,21.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.006001,7.714600e-07,None
3152,15,218,15218,2,152182,2018,2.364194e+04,6.530751,-72.774724,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008032,1.931910e-06,None
3153,25,297,25297,2,252972,2018,8.732733e+03,4.847916,-73.573646,10.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.004621,7.118800e-07,None


In [68]:

mapa_municipios = gpd.read_file('./shp_mapa_municipio/MGN_MPIO_POLITICO.shp')

In [71]:
mapa_df = mapa_municipios[mapa_municipios['MPIO_CDPMP'].isin(df['ciudad_municipio'].unique())]


In [109]:
mapa_df = mapa_df.merge(how='left', right=df[['ciudad_municipio','num_casos']].rename(columns={'ciudad_municipio':'MPIO_CDPMP'}), on='MPIO_CDPMP')

In [111]:
mapa = folium.Map(location=[4.3, -73.40], tiles='cartodbpositron', zoom_start=5.5, min_zoom=6, )

capa = folium.Choropleth(
    geo_data=mapa_df, 
    name="prueba1", 
    data=df,
    columns=['ciudad_municipio', 'num_casos'],
    key_on='feature.properties.MPIO_CDPMP',
    smooth_factor=0,
    fill_color='YlOrRd_09'
    )

capa.add_to(mapa)
capa.geojson.add_child(folium.features.GeoJsonTooltip(['MPIO_CNMBR', 'num_casos'], labels=False))

folium.LayerControl().add_to(mapa)

mapa.save('./assets/mapa_resumen.html')